In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import tqdm

In [2]:
L=3
N=2
H=4

In [13]:
def coord_func(x,L,h):
    r = np.random.random()
    print('r:',r)
    return x + 2*(r-0.5)/L**h

In [15]:
x_list = []
x=0.5
for h in range(1,H+1):
    x_list.append(coord_func(x,L,h))
    x = x_list[-1]
    print('x:',x)

r: 0.8487309390574734
x: 0.7324872927049823
r: 0.7315748147833355
x: 0.7839483626568345
r: 0.23258000331592332
x: 0.7641394740135696
r: 0.2600050282762464
x: 0.7582136722426127
